In [2]:
import pandas as pd
from collections import Counter
from Bio.Blast import NCBIXML

In [3]:
def xml2df(filepath):
    """convert BLAST xml output to pd.Dataframe."""
    
    dct_lst=[]
    #resultFilepath="/data/mitsuki/out/altorf/evolve/result/GCF_000010665.1_ASM1066v1.xml"
    with open(filepath) as f:
        blastRecords = NCBIXML.parse(f)
        for rec in blastRecords:
            queryName=rec.query
            category=queryName.split('|')[0]
            queryLength=rec.query_length
            for alignment in rec.alignments:
                sbjctName=alignment.title
                sbjctLength=alignment.length
                for hsp in alignment.hsps:
                    dct={}
                    dct["category"]=category
                    dct["query_name"]=queryName
                    dct["sbjct_name"]=sbjctName
                    dct["evalue"]=hsp.expect
                    dct["query_length"]=queryLength
                    dct["sbjct_length"]=sbjctLength
                    dct["query_start"]=hsp.query_start
                    dct["quer_end"]=hsp.query_end
                    dct["sbjct_start"]=hsp.sbjct_start
                    dct["sbjct_end"]=hsp.sbjct_end
                    dct_lst.append(dct)
                    
    result_df=pd.DataFrame(dct_lst)
    result_df=result_df[["category","query_name","sbjct_name","evalue"]]
    return result_df

In [4]:
def totalize_result(result_df):
    """totalize #hit for each category"""
    thres_lst=[1,0.1,0.01,1e-3,1e-4,1e-5]
    dct_lst=[]
    for thres in thres_lst:
        dct={}
        dct["thres"]=thres
        filtered_df=result_df[result_df["evalue"]<thres]
        #filtered_df=filtered_df.drop_duplicates(subset=['qseqid'])
        dct.update(Counter(filtered_df["category"]))
        dct_lst.append(dct)
    count_df=pd.DataFrame(dct_lst)
    
    count_df=count_df.fillna(0)
    name_lst=[]
    for prefix in ["F","DSF","DTSF"]:
        for i in range(2,7):
            name=prefix+str(i)
            name_lst.append(name)
    count_df[name_lst]=count_df[name_lst].astype(int)
    count_df=count_df[["thres"]+name_lst]
    
    return count_df

In [6]:
txtFilepath="../../createdatabase/out/genus_872.txt"
catalog_df=pd.read_csv(txtFilepath)
catalog_df.head()

,taxid,kingdom,phylum,class,order,family,genus,species,count_real,count_sim,diff,ftp_basename,organism_name,genetic_code,G+C
0,525146,-1,1224,28221,213115,194924,872,876,0.851789,0.777740,0.074048,GCF_000022125.1_ASM2212v1,Desulfovibrio desulfuricans subsp. desulfurica...,11,0.580722
1,883,-1,1224,28221,213115,194924,872,881,1.619893,1.407806,0.212088,GCF_000021385.1_ASM2138v1,Desulfovibrio vulgaris str. 'Miyazaki F',11,0.671090
2,901,-1,1224,28221,213115,194924,872,901,1.387866,1.183206,0.204661,GCF_900116045.1_DESPIGER,Desulfovibrio piger,11,0.641799
3,526222,-1,1224,28221,213115,194924,872,880,0.594646,0.536652,0.057994,GCF_000023445.1_ASM2344v1,Desulfovibrio salexigens DSM 2638,11,0.470928
4,641491,-1,1224,28221,213115,194924,872,876,1.622397,1.369687,0.252709,GCF_000189295.2_ASM18929v2,Desulfovibrio desulfuricans ND132,11,0.652094


In [7]:
for basename in catalog_df["ftp_basename"]:
    print()
    resultFilepath="/data/mitsuki/out/altorf/evolve/result/{}.xml".format(basename)
    print()
    print("PROCESSING {}".format(resultFilepath))
    result_df=xml2df(resultFilepath)
    count_df=totalize_result(result_df)
    print(count_df)



PROCESSING /data/mitsuki/out/altorf/evolve/result/GCF_000022125.1_ASM2212v1.xml
     thres   F2   F3    F4   F5   F6  DSF2  DSF3  DSF4  DSF5  DSF6  DTSF2  \
0  1.00000  637  684  1425  577  748   550   600  1143   466   606    521   
1  0.10000  102   98   194   80  101    83    80   149    67    68     64   
2  0.01000   19   24    36   15   18    18    15     9     7     7      5   
3  0.00100   12   10    10    3    3     2     5     1     1     1      1   
4  0.00010   11   10     7    3    2     0     1     0     0     0      0   
5  0.00001    8    8     4    3    2     0     1     0     0     0      0   

   DTSF3  DTSF4  DTSF5  DTSF6  
0    580   1245    450    675  
1     68    160     62     79  
2     13     22     13     13  
3      2      4      4      2  
4      0      1      1      0  
5      0      0      1      0  


PROCESSING /data/mitsuki/out/altorf/evolve/result/GCF_000021385.1_ASM2138v1.xml
     thres   F2    F3    F4   F5    F6  DSF2  DSF3  DSF4  DSF5  DSF6  DT

In [13]:
result_df["query_head"]=[name.split()[0] for name in result_df["query_name"]]

0              F3|lcl|NC_012796.1_cds_WP_012749587.1_2
1              F4|lcl|NC_012796.1_cds_WP_012749587.1_2
2              F3|lcl|NC_012796.1_cds_WP_012749588.1_3
3              F3|lcl|NC_012796.1_cds_WP_012749588.1_3
4              F3|lcl|NC_012796.1_cds_WP_012749588.1_3
5              F4|lcl|NC_012796.1_cds_WP_012749588.1_3
6              F6|lcl|NC_012796.1_cds_WP_012749588.1_3
7              F4|lcl|NC_012796.1_cds_WP_012749589.1_4
8              F4|lcl|NC_012796.1_cds_WP_012749590.1_5
9              F4|lcl|NC_012796.1_cds_WP_012749590.1_5
10             F6|lcl|NC_012796.1_cds_WP_012749590.1_5
11             F3|lcl|NC_012796.1_cds_WP_012749591.1_6
12             F6|lcl|NC_012796.1_cds_WP_012749591.1_6
13             F3|lcl|NC_012796.1_cds_WP_012749593.1_8
14            F2|lcl|NC_012796.1_cds_WP_012749595.1_10
15            F2|lcl|NC_012796.1_cds_WP_012749595.1_10
16            F4|lcl|NC_012796.1_cds_WP_012749595.1_10
17            F4|lcl|NC_012796.1_cds_WP_012749595.1_10
18        

In [16]:
result_df.shape

(21390, 5)

In [22]:
result_df[result_df["query_head"]=="F4|lcl|NC_012796.1_cds_WP_015862362.1_3577"]

,category,query_name,sbjct_name,evalue,query_head
6112,F4,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577 [ge...,gnl|BL_ORD_ID|32852 WP_081429644.1 DDE transpo...,7.646050e-08,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577
6113,F4,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577 [ge...,gnl|BL_ORD_ID|32877 WP_081429669.1 hypothetica...,2.111050e-07,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577
6114,F4,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577 [ge...,gnl|BL_ORD_ID|4245 WP_012613158.1 NusB/RsmB/TI...,3.163200e-01,F4|lcl|NC_012796.1_cds_WP_015862362.1_3577


In [4]:
resultFilepath="/data/mitsuki/out/altorf/evolve/result/GCF_000010665.1_ASM1066v1.xml"
result_df=xml2df(resultFilepath)
print(result_df.shape)
result_df.head()

(21390, 4)


,category,query_name,sbjct_name,evalue
0,F3,F3|lcl|NC_012796.1_cds_WP_012749587.1_2 [locus...,gnl|BL_ORD_ID|2770 WP_012611520.1 ribosomal la...,0.848814
1,F4,F4|lcl|NC_012796.1_cds_WP_012749587.1_2 [locus...,gnl|BL_ORD_ID|20189 WP_011367756.1 magnesium t...,0.015497
2,F3,F3|lcl|NC_012796.1_cds_WP_012749588.1_3 [locus...,gnl|BL_ORD_ID|8354 WP_015850381.1 HAD family h...,0.077864
3,F3,F3|lcl|NC_012796.1_cds_WP_012749588.1_3 [locus...,gnl|BL_ORD_ID|18721 WP_047170687.1 hypothetica...,0.342326
4,F3,F3|lcl|NC_012796.1_cds_WP_012749588.1_3 [locus...,gnl|BL_ORD_ID|22786 WP_062251346.1 hypothetica...,0.415651


In [6]:
resultFilepath="/data/mitsuki/out/altorf/evolve/result/GCF_000010665.1_ASM1066v1.xml"
blastRecords = NCBIXML.parse(open(resultFilepath))
for i,rec in enumerate(blastRecords):
    print(rec.query)
    if i>=11:
        break

F2|lcl|NC_012796.1_cds_WP_012749586.1_1 [locus_tag=DMR_RS00005] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_012749586.1] [location=126..1466] [gbkey=CDS]
F3|lcl|NC_012796.1_cds_WP_012749586.1_1 [locus_tag=DMR_RS00005] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_012749586.1] [location=126..1466] [gbkey=CDS]
F4|lcl|NC_012796.1_cds_WP_012749586.1_1 [locus_tag=DMR_RS00005] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_012749586.1] [location=126..1466] [gbkey=CDS]
F5|lcl|NC_012796.1_cds_WP_012749586.1_1 [locus_tag=DMR_RS00005] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_012749586.1] [location=126..1466] [gbkey=CDS]
F6|lcl|NC_012796.1_cds_WP_012749586.1_1 [locus_tag=DMR_RS00005] [protein=chromosomal replication initiator protein DnaA] [protein_id=WP_012749586.1] [location=126..1466] [gbkey=CDS]
F2|lcl|NC_012796.1_cds_WP_012749587.1_2 [locus_tag=DMR_RS00010] [protein=DNA polymerase II

In [7]:
for alignment in rec.alignments:
    sbjctName=alignment.title
    sbjctLength=alignment.length
    print(sbjctName)
    for hsp in alignment.hsps:
        print(hsp.query)
        print(hsp.match)
        print(hsp.sbjct)
    print()

gnl|BL_ORD_ID|8354 WP_015850381.1 HAD family hydrolase [Desulfovibrio salexigens]
HPGHGRRGERGEGSRKPGSPALPQDRHHDRRRRGRGAHPHPAFDLFLPALRKAHFKRLRLH-RPAAAVPRVQRRFRAFHQGRRGIVPVSHGPH
HP    R   G+G++K     LP+D+ +              +D F+P L K   + L  H RP A +P V   F A   G++ I  +S+ PH
HPVDAYRKFVGDGAKKLAWRVLPEDKQNQED-----------YDQFVPVLLKKFEEELNKHVRPYAGIPEVLADFIA--AGKK-IAILSNKPH

gnl|BL_ORD_ID|18721 WP_047170687.1 hypothetical protein [Desulfovibrio africanus]
PVGVSGSHGAPGRQEASPALRARRARHRLDRHRRGREHRHHHTVSARRGDLRDQPVQPRDPGQAVRGTGLPQPQAGAGF
PV V G   +      S  +R R A  RL R   GREH +  T       L   PVQ  +  +      L +  AG GF
PVDVDGQKTSQLSLRVSRKVRDRLAAARLARESYGREHGNEETEICLFAVLAGVPVQSIEELEIADYAKLQEAYAGDGF

gnl|BL_ORD_ID|22786 WP_062251346.1 hypothetical protein [Desulfovibrio fairfieldensis]
RGDLRDQPVQPRDPGQAVRGTGLPQPQAGAGFPRRADQRAGHLPLRRRPQSVRHRPQRRRAGHP
+GD+R      RD G+ + GT L          +R +  +G L L  RP SV + P    AG P
KGDIRAMTTLIRD-GKRLIGTKLQNTAVSEAVLQRGETVSGELQLLGRPYSVVYWPILNMAGKP



In [9]:
count_df=totalize_result(result_df)
count_df

,thres,F2,F3,F4,F5,F6,DSF2,DSF3,DSF4,DSF5,DSF6,DTSF2,DTSF3,DTSF4,DTSF5,DTSF6
0,1.00000,1430,1561,2697,831,1318,1120,1236,2314,675,1180,1094,1402,2491,727,1314
1,0.10000,284,269,418,131,213,202,165,293,92,148,172,176,340,99,145
2,0.01000,77,77,102,21,44,40,24,31,20,16,31,17,39,12,17
3,0.00100,35,36,57,9,26,4,2,1,1,0,6,3,2,1,3
4,0.00010,21,27,43,6,25,0,1,0,0,0,0,0,0,0,0
5,0.00001,17,23,41,3,25,0,0,0,0,0,0,0,0,0,0
